# Maximum Likelihood Estimation

In [1]:
import os
import argparse
import time
from datetime import datetime
import random
import sys
import atiSetup

from fit import runFits

Welcome to JupyROOT 6.28/06


In [2]:
############## SET ENVIRONMENT VARIABLES ##############
REPO_HOME = os.environ['REPO_HOME']
USE_MPI, USE_GPU, RANK_MPI = atiSetup.setup(globals())

cfgfile = f'{REPO_HOME}/tests/samples/SIMPLE_EXAMPLE/fitInit.cfg'
randomSeed = int(time.time())
seedfile = f'seed_file'
numRnd = 0

atiSetup| jupyter-lab called python3.9

------------------------------------------------
atiSetup| MPI is disabled
atiSetup| GPU is disabled
------------------------------------------------


atiSetup| Loading library libAmpTools.so ..............  ON
atiSetup| Loading library libAmpPlotter.so ............  ON
atiSetup| Loading library libAmpsDataIO.so ............  ON
atiSetup| Loading library libFSRoot.so ................  OFF
atiSetup| Loading library libAmpsGen.so ...............  OFF


------------------------------------------------
------------------------------------------------

Initializing AMPTOOLS_AMPS_DATAIO for PyROOT...


In [3]:
print("\n\n === COMMANDLINE ARGUMENTS === ")
print("Config file:", cfgfile)
print("Seed file:", seedfile)
print("Number of random fits:", numRnd)
print("Random seed:", randomSeed)
print(" ============================= \n\n")



 === COMMANDLINE ARGUMENTS === 
Config file: /w/halld-scshelf2101/lng/WORK/PyAmpTools/tests/samples/SIMPLE_EXAMPLE/fitInit.cfg
Seed file: seed_file
Number of random fits: 0
Random seed: 1699218519




We first load the configuration file by first passing it through a parser. We can display a detailed representation of its contents like so

In [4]:
parser = ConfigFileParser(cfgfile)
cfgInfo: ConfigurationInfo = parser.getConfigurationInfo()
cfgInfo.display()

   |        ^                                                      |
   |       / \             Version:  v0.15.1-dirty                 |
   |      /---\                                                    |
   |     /     \           GDouble:  8 bytes                       |
   |    /       \ MP           MPI:  NO                            |
   |     -------               GPU:  NO                            |
   |        |                                                      |
   |        |              doi.org/10.5281/zenodo.5039377          |
   |        | OOLS                                                 |

[ ConfigurationInfo ]:

	## CONFIGURATION INFO DISPLAY ##
	

[ ReactionInfo ]:

	############################################
	#############   REACTION INFO  #############
	############################################
	      REACTION NAME:  etapi
	      PARTICLE LIST:  4
			1.  Beam
			2.  Proton
			3.  Eta
			4.  Pi0
	        DATA READER:  ROOTDataReader
					/w/halld-scshel

The confirguation file specifies a datareader called `ROOTDataReader` and the amplitude called `Zlm`. There are many datareaders (i.e. for `FSRoot` and another for `Bootstrapping`) and amplitudes (i.e. `BreitWigner` and `Piecewise`). Below, `DataReader` is an alias for `ROOTDataReader` or its `MPI` implementation. These aliases are created by `atiSetup`.

In [5]:
AmpToolsInterface.registerAmplitude( Zlm() )
AmpToolsInterface.registerDataReader( DataReader() )
# AmpToolsInterface.registerAmplitude( BreitWigner() )
# AmpToolsInterface.registerAmplitude( Piecewise() )
# AmpToolsInterface.registerDataReader( DataReaderBootstrap() )

`AmpToolsInterface` is the primary way to interact with the `AmpTools` minimization library. From this `object` we can access things like a `ParameterManager` and a `MinimizationManager`. 

In [6]:
ati = AmpToolsInterface( cfgInfo )
AmpToolsInterface.setRandomSeed(randomSeed)


[ Minuit ]:

	**********
	 **    1 **SET STRATEGY  1
	**********
	 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY

[ AmplitudeManager ]:

	Creating AmplitudeManager for the reaction:  etapi
		 particle index assignment:  Beam -> 0
		 particle index assignment:  Proton -> 1
		 particle index assignment:  Eta -> 2
		 particle index assignment:  Pi0 -> 3

[ NormIntInterface ]:

	Loading generated Monte Carlo from file...
Resetting source kin in /w/halld-scshelf2101/lng/WORK/PyAmpTools/tests/samples/SIMPLE_EXAMPLE/samples/flat_generated_samples.root
	Duplicated Monte Carlo set detected, using previously loaded version


A `fit` module is defined in PyAmpTools to run the standard MLE fit. We simply pass the recently created `ati` instance and the number of fits with random initializations to perform. Randomization is beneficial as the optimizer could get stuck in unwanted local minima.

A single fit is performed without randomization is performed if `numRnd=0`

See [fit](https://lan13005.github.io/PyAmpTools/api/fit.html) for API and source code, including `runFits` below

In [7]:
nll = runFits(ati, N=numRnd)

LIKELIHOOD BEFORE MINIMIZATION: 112356.11732090596
LIKELIHOOD AFTER MINIMIZATION (NO RANDOMIZATION): 14346.408126566832
Resetting source kin in /w/halld-scshelf2101/lng/WORK/PyAmpTools/tests/samples/SIMPLE_EXAMPLE/samples/data_generated_samples.root

[ Minuit ]:

	**********
	 **    2 **MIGRAD 100000 0.1
	**********
	 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
	 START MIGRAD MINIMIZATION.  STRATEGY  1  CONVERGENCE WHEN EDM .LT. 1.00e-04
	 FCN=26585.6 FROM MIGRAD     STATUS=INITIATE      162         163 TOTAL
	                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
	  EXT PARAMETER                         CURRENT GUESS       STEP         FIRST   
	  NO.   NAME                  VALUE          ERROR          SIZE      DERIVATIVE 
	    1 etapi::reZ::resAmp1_re  -8.24991e+01   1.00000e-01       -92.499        75.351
	    2 etapi::reZ::resAmp2_re   9.26049e+01   1.00000e-01             0        2.0926
	    3 etapi::reZ::resAmp2_im   2.78287e+02   1.0

After the fit is complete, a `.fit` file will be created with the name defined by the `fit` keyword in the cfg file. This file contains the results of the fit. A tutorial on how to plot the results of these `.fit` files will be shown in 

In [8]:
!rm -f seed_0.txt
!rm -f normint
!rm -f result_0.fit